In [1]:
import numpy as np
import pandas as pd 
import folium 
from folium.plugins import FastMarkerCluster
import matplotlib.pyplot as plt
import matplotlib as mpl

This cell just loads the data from the original dataset, organizes them by time and gets rid of the rows that don't have either a latitude or a longitude

In [3]:
file_path = '../crashes.csv'
df = pd.read_csv(file_path)

df['datetime'] = pd.to_datetime((df["CRASH DATE"] + " " + df["CRASH TIME"]))
df.sort_values(by=["datetime"], inplace=True, ascending=False)
df = df.dropna(subset=['LATITUDE','LONGITUDE'])

/home/asifmallik/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


We then get rid of all of the columns that we don't need and save the data for later use. Moroever we reorganize it so that the data is in chronological order in the dataframe

In [ ]:
df_clean = df[['datetime','LONGITUDE','LATITUDE']]
df_clean.columns = ['datetime', 'longitude','latitude']
df_clean.reset_index(drop=True, inplace=True)
df_clean = df_clean.reindex(index=df_clean.index[::-1])
df_clean.reset_index(drop=True, inplace=True)
df_clean.to_csv('clean_data.csv',index=True)
df.head()

Now, we turn to aggregating the data. For doing so we will create a grid on top of new york city and will cluster the points such that each point is assigned to the square in the grid it belongs to. The representative of each square is simply going to be the center of the square. We illustrate this by plotting the points in new york city correponsing to one day 

In [143]:
df2013 = df_clean[(df_clean['datetime']>'2013-04-03') & (df_clean['datetime'] < '2013-04-04')]

In [144]:

def aggregate_data(df, lower_left, upper_right, n):
    """
    lower_left: lower left lat and long of grid of grids
    upper_right: upper left lat and long of grid of girds
    number of divisions
    """
    df_copy = pd.DataFrame.copy(df)
    
    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)
    
    xleft = lower_left[1]
    xright = upper_right[1]
    
    ybot = lower_left[0]
    yup = upper_right[0]
    
    xstride = (xright - xleft)/n
    ystride = (yup - ybot)/n
    
    df_copy['x_box'] = np.floor((df_copy['longitude'] - xleft)/xstride)
    df_copy['y_box'] = np.floor((df_copy['latitude'] - ybot)/ystride)
    df_copy
    df_copy['repr_lat'] = df_copy['y_box']* ystride + ystride/2 + ybot
    df_copy['repr_long'] = df_copy['x_box']* xstride + xstride/2 + xleft
    
    return df_copy
    
    
    


In [145]:
# These parameters were determined by looking at a map
lower_left = [49.68, -74.669]
upper_right = [59.145, 2.77]
location = [40.7128, -74.0060]
lower_left = [location[0] - 0.3, location[1] - 0.3]
upper_right = [location[0] + 0.3, location[1] + 0.4]


dfyear = aggregate_data(df2013, lower_left, upper_right, 600)

In [146]:
m = folium.Map()
m.add_child(FastMarkerCluster(dfyear[['repr_lat','repr_long']].values.tolist()))

,datetime,longitude,latitude,x_box,y_box,repr_lat,repr_long
125854,2013-04-03 00:01:00,-73.877473,40.839984,367.0,427.0,40.8403,-73.877250
125855,2013-04-03 00:11:00,-73.985283,40.645767,274.0,232.0,40.6453,-73.985750
125856,2013-04-03 00:27:00,-73.988355,40.720076,272.0,307.0,40.7203,-73.988083
125857,2013-04-03 00:30:00,-73.863096,40.750671,379.0,337.0,40.7503,-73.863250
125858,2013-04-03 00:50:00,-73.833443,40.771199,405.0,358.0,40.7713,-73.832917
...,...,...,...,...,...,...,...
126295,2013-04-03 23:15:00,-73.820299,40.755514,416.0,342.0,40.7553,-73.820083
126296,2013-04-03 23:15:00,-73.959452,40.653575,297.0,240.0,40.6533,-73.958917
126297,2013-04-03 23:18:00,-73.920128,40.661122,330.0,248.0,40.6613,-73.920417
126298,2013-04-03 23:40:00,-73.904131,40.638578,344.0,225.0,40.6383,-73.904083
